This notebook is based off of the example provided by hugging face.  

Referenced: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb


# Load Data and prepare features 

In [50]:
from datasets import load_dataset, load_metric 
from transformers import (AutoTokenizer,AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator)
import numpy as np

max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [1]:
# uncomment before running on google colab 
!pip install transformers==4.8.1
!pip install datasets 

     |████████████████████████████████| 2.5 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 43.7 MB/s 
     |████████████████████████████████| 895 kB 75.6 MB/s 
     |████████████████████████████████| 270 kB 5.1 MB/s 
     |████████████████████████████████| 243 kB 49.0 MB/s 
     |████████████████████████████████| 1.3 MB 45.3 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 119 kB 39.7 MB/s 
     |████████████████████████████████| 294 kB 60.7 MB/s 
     |████████████████████████████████| 142 kB 77.9 MB/s 
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.8.1 requires huggingface-hub==0.0.12, but y

In [3]:
# load dataset 
datasets = load_dataset("squad_v2")

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# instantiate tokenizer 
tokenizer = AutoTokenizer.from_pretrained('google/electra-small-discriminator')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [34]:
def prepare_train_features(examples):
    ''' given examples from Squad dataset: 
     accounts for examples longer then 512 tokens 
    returns: tokenized examples with: input ids, attention_mask, answer start position index, answer end position index''' 

    # tokenize examples accounting for some of them being to long to fit in a single feature 
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # get mapping from features to corresponding example in dataset  
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    #  get offset_mapping to map tokens to character position in original context  
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # add keys for start_positions and end_positions of answers to tokenized_examples 
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    # iterate through all offset_mappings (the corresponding start and end character in the original text that gave our token.)
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i] # input ids for tokenized examples 
        #cls_index = input_ids.index(tokenizer.cls_token_id)         # get cls index to label impossible answers 


        # Grab the sequence  corresponding for example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans,  
        sample_index = sample_mapping[i] # index of the example containing this span of text.
        answers = examples["answers"][sample_index] # get answers for this example 
        # If no answers are given for this example set the answer start and end position to 0 
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(0)
            tokenized_examples["end_positions"].append(0)
        else: # answers are given 
            # get start and end character index of the answer in the context.
            start_char_index = answers["answer_start"][0]
            end_char_index = start_char_index + len(answers["text"][0])

            # get start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # get end token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # if the answer is out of the span 
            if not (offsets[token_start_index][0] <= start_char_index and offsets[token_end_index][1] >= end_char_index):
              #  set start and end position for answer to 0 since not in this span
                tokenized_examples["start_positions"].append(0)
                tokenized_examples["end_positions"].append(0)
            else: # the answer is in the span 
            # Move the token_start_index and token_end_index to the two ends of the answer.
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char_index: 
                    token_start_index += 1 # keep shifting to the right until token_start_index refers to where the answer starts in text 
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char_index: # start on right side and work down
                    token_end_index -= 1 # keep shifting left until token_end_index points to where answer ends in text 
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [35]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

# Train 

In [ ]:
# check which gpu connected to 
!nvidia-smi

In [49]:
# instantiate pretrained model for finetuning  
model = AutoModelForQuestionAnswering.from_pretrained('google/electra-small-discriminator')

Downloading:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['qa_outputs.weight', 'qa_output

In [52]:
# instantiate data collator 
data_collator = default_data_collator 

# arguments for trainer 
args = TrainingArguments(
    f"drive/MyDrive/electra-small-squad_v2_run1",
    #evaluation_strategy = "steps",
    #max_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy= 'epoch',)


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer= tokenizer,    
    
)

In [ ]:
trainer.train()


***** Running training *****
  Num examples = 130503
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 20395


Step,Training Loss
500,3.452100
1000,2.363400
1500,1.982000
2000,1.780400
2500,1.659100
3000,1.584000
3500,1.518200
4000,1.449800
4500,1.373800
5000,1.357800


Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run1/checkpoint-4079
Configuration saved in drive/MyDrive/electra-small-squad_v2_run1/checkpoint-4079/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run1/checkpoint-4079/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run1/checkpoint-4079/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run1/checkpoint-4079/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run1/checkpoint-8158
Configuration saved in drive/MyDrive/electra-small-squad_v2_run1/checkpoint-8158/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run1/checkpoint-8158/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run1/checkpoint-8158/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run1/checkpoint-8158/special_tokens_map.json
Sa

TrainOutput(global_step=20395, training_loss=1.2795918428421489, metrics={'train_runtime': 7982.9204, 'train_samples_per_second': 81.739, 'train_steps_per_second': 2.555, 'total_flos': 2.702807149925376e+16, 'train_loss': 1.2795918428421489, 'epoch': 5.0})

In [ ]:
trainer.save_model('drive/MyDrive/electra_5e.pt')

Saving model checkpoint to drive/MyDrive/electra_5e.pt
Configuration saved in drive/MyDrive/electra_5e.pt/config.json
Model weights saved in drive/MyDrive/electra_5e.pt/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra_5e.pt/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra_5e.pt/special_tokens_map.json


# 2nd run 

In [ ]:
# load model from last checkpoint 
model_5e = AutoModelForQuestionAnswering.from_pretrained('drive/MyDrive/electra_5e.pt')
tokenizer_5e = AutoTokenizer.from_pretrained('drive/MyDrive/electra_5e.pt')
# tokenize dataset for training 
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

args_run_2 = TrainingArguments(
    f"drive/MyDrive/electra-small-squad_v2_run2",
    #evaluation_strategy = "steps",
    #max_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=50,
    weight_decay=0.01,
    save_strategy= 'epoch',
)

trainer_run2 = Trainer(
    model_5e, 
    args_run_2,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer= tokenizer_5e,    
    
)

loading configuration file drive/MyDrive/electra_5e.pt/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.8.1",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

loading weights file drive/MyDrive/electra_5e.pt/pytorch_model.bin
All model checkpoint weights were used when initializing ElectraForQuestionAnswering.

All the weights of ElectraFo

In [ ]:
trainer_run2.train()

***** Running training *****
  Num examples = 130503
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 203950


Step,Training Loss
500,1.001800
1000,0.994500
1500,0.982200
2000,0.957600
2500,0.942100
3000,0.941700
3500,0.932800
4000,0.909600
4500,0.841600
5000,0.846800


Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run2/checkpoint-4079
Configuration saved in drive/MyDrive/electra-small-squad_v2_run2/checkpoint-4079/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run2/checkpoint-4079/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run2/checkpoint-4079/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run2/checkpoint-4079/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run2/checkpoint-8158
Configuration saved in drive/MyDrive/electra-small-squad_v2_run2/checkpoint-8158/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run2/checkpoint-8158/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run2/checkpoint-8158/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run2/checkpoint-8158/special_tokens_map.json
Sa

In [ ]:
trainer_run2.save_model('drive/MyDrive/electra_55e.pt')

# 3rd run 

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments,Trainer, default_data_collator
model_r3 = AutoModelForQuestionAnswering.from_pretrained('drive/MyDrive/checkpoint-57106')
tokenizer = AutoTokenizer.from_pretrained('drive/MyDrive/checkpoint-57106')
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

args_run_3 = TrainingArguments(
    f"drive/MyDrive/electra-small-squad_v2_run3",
    #evaluation_strategy = "steps",
    #max_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy= 'epoch',
)

trainer_run3 = Trainer(
    model_r3, 
    args_run_3,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer= tokenizer,    
    
)

loading configuration file drive/MyDrive/checkpoint-57106/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.8.1",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

loading weights file drive/MyDrive/checkpoint-57106/pytorch_model.bin
All model checkpoint weights were used when initializing ElectraForQuestionAnswering.

All the weights of ElectraFor

In [ ]:
trainer_run3.train()

***** Running training *****
  Num examples = 130503
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 40790


Step,Training Loss
500,0.311800
1000,0.313000
1500,0.315900
2000,0.320600
2500,0.315300
3000,0.317800
3500,0.324500
4000,0.320700
4500,0.273600
5000,0.277300


Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run3/checkpoint-4079
Configuration saved in drive/MyDrive/electra-small-squad_v2_run3/checkpoint-4079/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run3/checkpoint-4079/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run3/checkpoint-4079/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run3/checkpoint-4079/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158
Configuration saved in drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/special_tokens_map.json
Sa

TrainOutput(global_step=40790, training_loss=0.2187850300311224, metrics={'train_runtime': 16053.6724, 'train_samples_per_second': 81.292, 'train_steps_per_second': 2.541, 'total_flos': 5.405614299850752e+16, 'train_loss': 0.2187850300311224, 'epoch': 10.0})

# run 4 

In [ ]:
model_r4 = AutoModelForQuestionAnswering.from_pretrained('drive/MyDrive/run3-checkpoint-40790')
tokenizer = AutoTokenizer.from_pretrained('drive/MyDrive/run3-checkpoint-40790')
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

args_run_4 = TrainingArguments(
    f"drive/MyDrive/electra-small-squad_v2_run4",
    #evaluation_strategy = "steps",
    #max_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy= 'epoch',
)

trainer_run4 = Trainer(
    model_r4, 
    args_run_4,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer= tokenizer,    
    
)

loading configuration file drive/MyDrive/run3-checkpoint-40790/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.8.1",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

loading weights file drive/MyDrive/run3-checkpoint-40790/pytorch_model.bin
All model checkpoint weights were used when initializing ElectraForQuestionAnswering.

All the weights 

  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer_run4.train()

***** Running training *****
  Num examples = 130503
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 40790


Step,Training Loss
500,0.120000
1000,0.120600
1500,0.127100
2000,0.132600
2500,0.126800
3000,0.131800
3500,0.135100
4000,0.142200
4500,0.118800
5000,0.122800


Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079
Configuration saved in drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158
Configuration saved in drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/special_tokens_map.json
Sa

TrainOutput(global_step=40790, training_loss=0.11120225214788686, metrics={'train_runtime': 16034.0753, 'train_samples_per_second': 81.391, 'train_steps_per_second': 2.544, 'total_flos': 5.405614299850752e+16, 'train_loss': 0.11120225214788686, 'epoch': 10.0})

# Run 5 


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments,Trainer, default_data_collator
model_r5 = AutoModelForQuestionAnswering.from_pretrained('drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790')
tokenizer = AutoTokenizer.from_pretrained('drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790')
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

args_run_5 = TrainingArguments(
    f"drive/MyDrive/electra-small-squad_v2_run5",
    #evaluation_strategy = "steps",
    #max_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy= 'epoch',
)

trainer_run5 = Trainer(
    model_r5, 
    args_run_5,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer= tokenizer,    
    
)

loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.8.1",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

loading weights file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790/pytorch_model.bin
All model checkpoint weights were used when initiali

  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer_run5.train()

***** Running training *****
  Num examples = 130503
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 40790


Step,Training Loss
500,0.036600
1000,0.045700
1500,0.048600
2000,0.055100
2500,0.057900
3000,0.058600
3500,0.058900
4000,0.065100
4500,0.066800
5000,0.057100


Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079
Configuration saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/special_tokens_map.json


Step,Training Loss
500,0.036600
1000,0.045700
1500,0.048600
2000,0.055100
2500,0.057900
3000,0.058600
3500,0.058900
4000,0.065100
4500,0.066800
5000,0.057100


Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run5/checkpoint-8158
Configuration saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-8158/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-8158/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-8158/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-8158/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run5/checkpoint-12237
Configuration saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-12237/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-12237/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-12237/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run5/checkpoint-12237/special_tokens_map.js

TrainOutput(global_step=40790, training_loss=0.06777338325816587, metrics={'train_runtime': 16067.3573, 'train_samples_per_second': 81.222, 'train_steps_per_second': 2.539, 'total_flos': 5.405614299850752e+16, 'train_loss': 0.06777338325816587, 'epoch': 10.0})

# run 6


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments,Trainer, default_data_collator
model_r6 = AutoModelForQuestionAnswering.from_pretrained('drive/MyDrive/electra-small-squad_v2_run5/checkpoint-40790')
tokenizer = AutoTokenizer.from_pretrained('drive/MyDrive/electra-small-squad_v2_run5/checkpoint-40790')
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

args_run_6 = TrainingArguments(
    f"drive/MyDrive/electra-small-squad_v2_run6",
    #evaluation_strategy = "steps",
    #max_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy= 'epoch',
    #evaluation_strategy='steps',
    #eval_steps=1
)

trainer_run6 = Trainer(
    model_r6, 
    args_run_6,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer= tokenizer,  
    #compute_metrics=compute_metrics  
    
)

loading configuration file drive/MyDrive/electra-small-squad_v2_run5/checkpoint-40790/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.8.1",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

loading weights file drive/MyDrive/electra-small-squad_v2_run5/checkpoint-40790/pytorch_model.bin
All model checkpoint weights 

In [ ]:
trainer_run6.train()

***** Running training *****
  Num examples = 130503
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 40790


Step,Training Loss
500,0.244700
1000,0.231200
1500,0.236000
2000,0.230800
2500,0.226900
3000,0.238900
3500,0.232500
4000,0.244300
4500,0.203900
5000,0.200000


Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079
Configuration saved in drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/electra-small-squad_v2_run6/checkpoint-8158
Configuration saved in drive/MyDrive/electra-small-squad_v2_run6/checkpoint-8158/config.json
Model weights saved in drive/MyDrive/electra-small-squad_v2_run6/checkpoint-8158/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/electra-small-squad_v2_run6/checkpoint-8158/tokenizer_config.json
Special tokens file saved in drive/MyDrive/electra-small-squad_v2_run6/checkpoint-8158/special_tokens_map.json
Sa

TrainOutput(global_step=40790, training_loss=0.15624639955797917, metrics={'train_runtime': 25679.3311, 'train_samples_per_second': 50.82, 'train_steps_per_second': 1.588, 'total_flos': 5.405614299850752e+16, 'train_loss': 0.15624639955797917, 'epoch': 10.0})

# run 7


In [ ]:
from transformers import default_data_collator
import numpy as np
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments,Trainer, default_data_collator

data_collator = default_data_collator
model_r7 = AutoModelForQuestionAnswering.from_pretrained('drive/MyDrive/electra-small-squad_v2_run6/checkpoint-40790')
tokenizer = AutoTokenizer.from_pretrained('drive/MyDrive/electra-small-squad_v2_run6/checkpoint-40790')
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

args_run_7 = TrainingArguments(
    f"drive/MyDrive/electra-small-squad_v2_run7",
    #evaluation_strategy = "epoch",
    #max_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy= 'epoch',
    evaluation_strategy='steps',
    eval_steps=500,
    prediction_loss_only=True
)

trainer_run7 = Trainer(
    model_r7, 
    args_run_7,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"].shuffle(seed=42).select(range(1000)),
    data_collator=data_collator,
    tokenizer= tokenizer,    
)

trainer_run7.train()

# Run 8 



In [ ]:
from transformers import default_data_collator
import numpy as np
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments,Trainer, default_data_collator

checkpoint= 'drive/MyDrive/electra-small-squad_v2_run7/checkpoint-20395'

data_collator = default_data_collator
model_r8 = AutoModelForQuestionAnswering.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

args_run_8 = TrainingArguments(
    f"drive/MyDrive/electra-small-squad_v2_run8",
    #evaluation_strategy = "epoch",
    #max_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy= 'epoch',
    #evaluation_strategy='steps',
    #eval_steps=500,
    #prediction_loss_only=True
)

trainer_run8 = Trainer(
    model_r8, 
    args_run_8,
    train_dataset=tokenized_datasets["train"],
    #eval_dataset=tokenized_datasets["validation"].shuffle(seed=42).select(range(1000)),
    data_collator=data_collator,
    tokenizer= tokenizer,    
)

trainer_run8.train()